In [101]:
# Libraries
import requests
import json
import pandas as pd

In [107]:
# Dictionary of consumed APIs  
API_DICT = {
    "categories": "https://api.mercadolibre.com/sites/MLA/categories",
    "lower_categories": "https://api.mercadolibre.com/categories/",
    "search": "https://api.mercadolibre.com/sites/MLA/search?category=",  
    "items": "https://api.mercadolibre.com/items?ids="
}

In [60]:
# Useful functions
def retrieve_data_from_api(url):
    "Returns a dictionary storing data from an API call"
    response = requests.get(url)
    data = response.text
    parsed = json.loads(data)
    return parsed

def flatten_and_append(dataframe, dictionary):
    """Flattens nested dictionary to achieve tabular format
    and appends it as a row in the given dataframe.
    """
    row = pd.json_normalize(dictionary)
    dataframe = dataframe.append(row)
    return dataframe    

In [61]:
# Load categories
categories_api = retrieve_data_from_api(APIs["categories"])
categories = pd.DataFrame.from_dict(categories_api)

In [82]:
# Load listed items for all categories
items = pd.DataFrame()

for category in list(categories.id):
    # Retrieve a list of items for each category
    search_api = retrieve_data_from_api(APIs["search"]+category)
    for item_dict in search_api["results"]:
        items = flatten_and_append(items, item_dict)

In [95]:
# Use items API to retrieve more features of the listed items
items_extended = pd.DataFrame()

for item in list(items.id):
    items_api = retrieve_data_from_api(APIs["items"]+item)
    items_extended = flatten_and_append(items_extended, items_api[0]["body"])

In [108]:
# Retrieve lower level categories for listed items
lower_categories = pd.DataFrame()
category_list = list(items.category_id.unique())

for category in category_list:
    lower_categories_api = retrieve_data_from_api(APIs["lower_categories"]+category)
    lower_categories = flatten_and_append(lower_categories, lower_categories_api)

In [ ]:
# Persist raw datasets
categories.to_csv("../data/raw/categories.txt", index=False)
lower_categories.to_csv("../data/raw/lower_categories.txt", index=False)
items.to_csv("../data/raw/items.txt", index=False)
items_extended.to_csv("../data/raw/items_extended.txt", index=False)